# Análise de Dados Experimentais - Vol 1

## Fundamentos de Estatística e Estimação de Parâmetros

### Márcio Schwaab e José Carlos Pinto

### Cap 5: Estimação de Parâmetros

### Exercício 5.1

Afrânio

[github.com/afraeq](github.com/afraeq)

[afrjr.weebly.com](afrjr.weebly.com)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from pyswarm import pso

colorcycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

#### Dados experimentais

In [2]:
xe = np.array([0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 7.0, 10.0])
ye = np.array([7.92, 18.51, 20.09, 18.97, 26.67, 29.45, 32.58, 34.54, 34.62])

sigma2 = np.array([25, 25, 9, 9, 1, 1, 0.25, 0.25, 0.01])

dados = np.array([xe, ye, sigma2]).T

NE = dados.shape[0] # Número de Experimentos

dados

array([[5.000e-01, 7.920e+00, 2.500e+01],
       [1.000e+00, 1.851e+01, 2.500e+01],
       [1.500e+00, 2.009e+01, 9.000e+00],
       [2.000e+00, 1.897e+01, 9.000e+00],
       [3.000e+00, 2.667e+01, 1.000e+00],
       [4.000e+00, 2.945e+01, 1.000e+00],
       [5.000e+00, 3.258e+01, 2.500e-01],
       [7.000e+00, 3.454e+01, 2.500e-01],
       [1.000e+01, 3.462e+01, 1.000e-02]])

#### Modelo

$$ y = \alpha_1(1-\exp(-\alpha_2x))$$

In [3]:
y = lambda x, alpha: alpha[0]*(1-np.exp(-alpha[1]*x))

#### Função objetivo

$$F_{obj} = \sum_{i=1}^{NE} \frac{(y_i^e-y_i^m)^2}{\sigma_i^2}$$

In [4]:
F_obj = lambda alpha: np.sum((ye-y(xe, alpha))**2/sigma2)

#### Minimizando função objetivo com o Enxame

In [5]:
lb = [0, 0]
ub = [50, 1]

alpha_opt, fopt = pso(F_obj, lb, ub)
print(alpha_opt, '\n', fopt)

Stopping search: Swarm best objective change less than 1e-08
[34.8527252  0.5128325] 
 6.075445407266264


#### Matriz de covariâncias dos parâmetros

$$\mathbf{V_{\alpha}} = [\mathbf{B^TV_y^{-1}B]^{-1}},$$

sendo:

* $\mathbf{B} = \displaystyle\left[\frac{\partial \mathbf{y_m}}{\partial \boldsymbol\alpha}\right]$, de dimensão NE $\times$ NP, a matriz de sensitividades;
* $\mathbf{V_y}$ a matriz de covariâncias das medidas experimentais, que resulta diagonal sob a hipótese de erros independentes.

In [6]:
# número de parâmetros
NP = 2

# desvio para cálculo das derivadas numéricas em B
dp = 1e-9

# matriz de covariâncias dos erros experimentais
Vy = np.diag(sigma2)

# matriz de sensitividades
B = np.zeros((NE, NP))
for i in range(NP):
    dev = np.zeros(NP)
    dev[i] = dp
    B[:,i] = (y(xe, alpha_opt + dev) - y(xe, alpha_opt - dev))/(2*dp)
    
# matriz de covariâncias dos parâmetros
V_alpha = np.linalg.inv(B.T@np.linalg.inv(Vy)@B)
V_alpha

array([[ 0.01568773, -0.00215721],
       [-0.00215721,  0.00072908]])

#### Desvios-padrões dos parâmetros

In [7]:
sigma_alpha = np.array([np.sqrt(V_alpha[i,i]) for i in range(NP)])
sigma_alpha

array([0.12525068, 0.02700148])

#### Coeficiente de correlação paramétrica

In [8]:
rho_alpha = V_alpha[0,1]/(sigma_alpha[0]*sigma_alpha[1])
rho_alpha

-0.6378605151162926

#### Intervalo de confiança dos parâmetros

Considerando que os erros paramétricos seguem a distribuição normal:

In [30]:
conf = [0.025, 0.975]

alpha_bounds = [[scipy.stats.norm.ppf(j, loc=alpha_opt[i], scale=sigma_alpha[i])
                for i in range(NP)] for j in conf]

print('alpha_1 = ',alpha_opt[0])
print(alpha_bounds[0][0],'< alpha_1 <',alpha_bounds[1][0],'\n')

print('alpha_2 = ',alpha_opt[1])
print(alpha_bounds[0][1],'< alpha_2 <',alpha_bounds[1][1])

alpha_1 =  34.852725197547144
34.607238369320946 < alpha_1 < 35.09821202577334 

alpha_2 =  0.5128325039716091
0.45991057331943347 < alpha_2 < 0.5657544346237847


#### Verificação da qualidade do modelo

In [10]:
chi2_inf = scipy.stats.chi2.ppf(0.025, NE-NP)
chi2_sup = scipy.stats.chi2.ppf(0.975, NE-NP)

print('F_obj = ',fopt)
print(chi2_inf,'< Chi2 <', chi2_sup)

F_obj =  6.075445407266264
1.689869180677355 < Chi2 < 16.012764274629326


In [29]:
ym = y(xe, alpha_opt)

res = ye - ym

res_mean = np.mean(res)
res_std = np.std(res, ddof = 1)

conf = [0.025, 0.975]

res_mean_bounds = [scipy.stats.norm.ppf(j, loc=res_mean, scale=res_std/np.sqrt(NE))
                   for j in conf]

print('res_mean = ',res_mean)
print(res_mean_bounds[0],'< res_mean <', res_mean_bounds[1])

res_mean =  0.21961259479161044
-1.1542863644836272 < res_mean < 1.5935115540668479
